# Wind Turbine Module

Models on-site wind electricity generation from kinetic energy of wind. Includes power curve modeling and exergy analysis.

## Energy Analysis

**Available wind power:**
$$\dot{E}_{wind,in} = \frac{1}{2} \rho A_{rotor} v^3$$

**Electrical power output:**
$$\dot{W}_{elec,wt} = C_p \cdot \dot{E}_{wind,in}$$

**Betz limit:** $C_{p,max} = \frac{16}{27} \approx 0.593$

Modern turbines: $C_p \approx 0.45 - 0.50$

**Energy efficiency:**
$$\eta_{en,wt} = \frac{\dot{W}_{elec,wt}}{\dot{E}_{wind,in}} = C_p$$

## Exergy Analysis

**Wind exergy input (kinetic exergy = kinetic energy):**
$$\dot{Ex}_{wind,in} = \dot{E}_{wind,in} = \frac{1}{2} \rho A_{rotor} v^3$$

**Electrical exergy output:**
$$\dot{Ex}_{elec,wt} = \dot{W}_{elec,wt}$$

**Exergy efficiency:**
$$\Psi_{wt} = \frac{\dot{Ex}_{elec,wt}}{\dot{Ex}_{wind,in}} = C_p$$

**Exergy destruction:**
$$\dot{Ex}_{d,wt} = \dot{Ex}_{wind,in} - \dot{Ex}_{elec,wt}$$

Note: For wind, $\Psi_{wt} = \eta_{en,wt} = C_p$ since kinetic exergy equals kinetic energy.

## Parameters

In [1]:
# Wind Turbine Default Parameters
const WT_DEFAULTS = (
    ρ_air = 1.225,          # Air density at sea level [kg/m³]
    C_p = 0.45,             # Power coefficient [-]
    v_cutin = 3.0,          # Cut-in wind speed [m/s]
    v_rated = 12.0,         # Rated wind speed [m/s]
    v_cutout = 25.0,        # Cut-out wind speed [m/s]
    BETZ_LIMIT = 16/27      # Theoretical maximum C_p
)

(ρ_air = 1.225, C_p = 0.45, v_cutin = 3.0, v_rated = 12.0, v_cutout = 25.0, BETZ_LIMIT = 0.5925925925925926)

## Core Functions

In [2]:
"""
    WindTurbine

Struct holding wind turbine model parameters.
"""
struct WindTurbine
    A_rotor::Float64        # Rotor swept area [m²]
    P_rated::Float64        # Rated power [W]
    C_p::Float64            # Power coefficient [-]
    ρ::Float64              # Air density [kg/m³]
    v_cutin::Float64        # Cut-in speed [m/s]
    v_rated::Float64        # Rated speed [m/s]
    v_cutout::Float64       # Cut-out speed [m/s]
    
    function WindTurbine(A_rotor::Float64, P_rated::Float64;
            C_p::Float64 = 0.45,
            ρ::Float64 = 1.225,
            v_cutin::Float64 = 3.0,
            v_rated::Float64 = 12.0,
            v_cutout::Float64 = 25.0)
        @assert C_p <= 16/27 "C_p cannot exceed Betz limit (0.593)"
        new(A_rotor, P_rated, C_p, ρ, v_cutin, v_rated, v_cutout)
    end
end

# Constructor from rotor diameter
function WindTurbine(; D_rotor::Float64, P_rated::Float64, kwargs...)
    A_rotor = π * (D_rotor/2)^2
    WindTurbine(A_rotor, P_rated; kwargs...)
end

WindTurbine

In [3]:
"""
    wind_power_available(wt::WindTurbine, v::Float64) -> Float64

Calculate available wind power [W] at given wind speed [m/s].
"""
function wind_power_available(wt::WindTurbine, v::Float64)
    return 0.5 * wt.ρ * wt.A_rotor * v^3
end

wind_power_available

In [4]:
"""
    wt_power_output(wt::WindTurbine, v::Float64) -> Float64

Calculate electrical power output [W] using power curve.
Accounts for cut-in, rated, and cut-out speeds.
"""
function wt_power_output(wt::WindTurbine, v::Float64)
    # Below cut-in or above cut-out: no power
    if v < wt.v_cutin || v > wt.v_cutout
        return 0.0
    end
    
    # Between cut-in and rated: cubic relationship
    if v < wt.v_rated
        P_avail = wind_power_available(wt, v)
        return min(wt.C_p * P_avail, wt.P_rated)
    end
    
    # Between rated and cut-out: constant rated power
    return wt.P_rated
end

wt_power_output

In [5]:
"""
    wt_efficiency(wt::WindTurbine, v::Float64) -> Float64

Calculate instantaneous energy efficiency [-].
"""
function wt_efficiency(wt::WindTurbine, v::Float64)
    P_avail = wind_power_available(wt, v)
    if P_avail <= 0.0
        return 0.0
    end
    P_out = wt_power_output(wt, v)
    return P_out / P_avail
end

wt_efficiency

In [6]:
"""
    analyze_wt(wt::WindTurbine, v::Float64) -> NamedTuple

Complete energy and exergy analysis for given wind speed.
v: wind speed [m/s]
"""
function analyze_wt(wt::WindTurbine, v::Float64)
    # Energy analysis
    E_wind_in = wind_power_available(wt, v)
    W_elec = wt_power_output(wt, v)
    η_en = E_wind_in > 0 ? W_elec / E_wind_in : 0.0
    
    # Exergy analysis (kinetic exergy = kinetic energy)
    Ex_wind_in = E_wind_in
    Ex_elec = W_elec
    Ex_d = Ex_wind_in - Ex_elec
    Ψ_wt = η_en  # For wind: exergy eff = energy eff
    
    return (
        v = v,                    # Wind speed [m/s]
        E_wind_in = E_wind_in,    # Available wind power [W]
        W_elec = W_elec,          # Electrical output [W]
        Ex_wind_in = Ex_wind_in,  # Wind exergy input [W]
        Ex_elec = Ex_elec,        # Electrical exergy [W]
        Ex_d = Ex_d,              # Exergy destruction [W]
        η_en = η_en,              # Energy efficiency [-]
        Ψ_wt = Ψ_wt,              # Exergy efficiency [-]
        C_p_actual = η_en         # Actual power coefficient [-]
    )
end

analyze_wt

## Tests

In [8]:
using Test

function run_wt_tests()
    println("Running Wind Turbine Tests...")
    
    # Create test turbine: 10m diameter, 5kW rated
    wt = WindTurbine(D_rotor=10.0, P_rated=5000.0, C_p=0.45)
    
    # Test 1: Rotor area calculation
    @testset "Rotor Area" begin
        @test wt.A_rotor ≈ π * 25.0 atol=1e-6  # π * r²
    end
    
    # Test 2: Betz limit enforcement
    @testset "Betz Limit" begin
        @test wt.C_p <= 16/27
        @test_throws AssertionError WindTurbine(50.0, 5000.0, C_p=0.6)
    end
    
    # Test 3: Cubic wind power relationship
    @testset "Cubic Relationship" begin
        P1 = wind_power_available(wt, 5.0)
        P2 = wind_power_available(wt, 10.0)
        @test P2 / P1 ≈ 8.0 atol=1e-6  # (10/5)³ = 8
    end
    
    # Test 4: Power curve regions
    @testset "Power Curve" begin
        # Below cut-in
        @test wt_power_output(wt, 2.0) == 0.0
        # Above cut-out
        @test wt_power_output(wt, 30.0) == 0.0
        # At rated speed: should be at or near rated power
        P_rated_speed = wt_power_output(wt, wt.v_rated)
        @test P_rated_speed ≈ wt.P_rated atol=100.0
        # Above rated: constant at rated
        @test wt_power_output(wt, 15.0) == wt.P_rated
    end
    
    # Test 5: Efficiency in cubic region (before hitting rated power)
    @testset "Efficiency" begin
        # Use low wind speed where C_p*P_avail < P_rated
        v = 5.0  # Low enough to not hit rated cap
        P_avail = wind_power_available(wt, v)
        P_out = wt_power_output(wt, v)
        # Verify we're not power-limited
        @test P_out < wt.P_rated
        # In cubic region, η should equal C_p
        η = wt_efficiency(wt, v)
        @test η ≈ wt.C_p atol=0.01
        @test η <= 16/27  # Cannot exceed Betz
    end
    
    # Test 6: Exergy balance
    @testset "Exergy Balance" begin
        result = analyze_wt(wt, 5.0)  # Use speed in cubic region
        # Exergy balance: Ex_in = Ex_out + Ex_d
        @test result.Ex_wind_in ≈ result.Ex_elec + result.Ex_d atol=1e-6
        # For wind: Ψ = η_en
        @test result.Ψ_wt ≈ result.η_en atol=1e-6
        @test result.Ex_d > 0  # Must have destruction
    end
    
    # Test 7: Zero wind
    @testset "Zero Wind" begin
        result = analyze_wt(wt, 0.0)
        @test result.W_elec == 0.0
        @test result.E_wind_in == 0.0
    end
    
    # Test 8: Power-limited region
    @testset "Power Limited" begin
        # At high wind, efficiency drops below C_p due to rated cap
        v = 10.0
        η = wt_efficiency(wt, v)
        @test η < wt.C_p  # Efficiency limited by P_rated
    end
    
    println("All tests passed!")
end

run_wt_tests()

Running Wind Turbine Tests...
Test Summary: | Pass  Total  Time
Rotor Area    |    1      1  0.0s
Test Summary: | Pass  Total  Time
Betz Limit    |    2      2  0.0s
Test Summary:      | Pass  Total  Time
Cubic Relationship |    1      1  0.0s
Test Summary: | Pass  Total  Time
Power Curve   |    4      4  0.0s
Test Summary: | Pass  Total  Time
Efficiency    |    3      3  0.0s
Test Summary:  | Pass  Total  Time
Exergy Balance |    3      3  0.0s
Test Summary: | Pass  Total  Time
Zero Wind     |    2      2  0.0s
Test Summary: | Pass  Total  Time
Power Limited |    1      1  0.0s
All tests passed!


## Example Usage

In [9]:
# Example: 10m diameter turbine, 5kW rated, at 8 m/s wind
wt = WindTurbine(D_rotor=10.0, P_rated=5000.0, C_p=0.45)
result = analyze_wt(wt, 8.0)

println("Wind Turbine Analysis")
println("="^40)
println("Rotor diameter:     $(round(sqrt(wt.A_rotor/π)*2, digits=1)) m")
println("Rotor area:         $(round(wt.A_rotor, digits=1)) m²")
println("Rated power:        $(wt.P_rated/1000) kW")
println("Wind speed:         $(result.v) m/s")
println("─"^40)
println("Available wind:     $(round(result.E_wind_in/1000, digits=2)) kW")
println("Electrical output:  $(round(result.W_elec/1000, digits=2)) kW")
println("Energy efficiency:  $(round(result.η_en*100, digits=1))%")
println("─"^40)
println("Wind exergy in:     $(round(result.Ex_wind_in/1000, digits=2)) kW")
println("Exergy destruction: $(round(result.Ex_d/1000, digits=2)) kW")
println("Exergy efficiency:  $(round(result.Ψ_wt*100, digits=1))%")
println("─"^40)
println("Betz limit:         $(round(16/27*100, digits=1))%")

Wind Turbine Analysis
Rotor diameter:     10.0 m
Rotor area:         78.5 m²
Rated power:        5.0 kW
Wind speed:         8.0 m/s
────────────────────────────────────────
Available wind:     24.63 kW
Electrical output:  5.0 kW
Energy efficiency:  20.3%
────────────────────────────────────────
Wind exergy in:     24.63 kW
Exergy destruction: 19.63 kW
Exergy efficiency:  20.3%
────────────────────────────────────────
Betz limit:         59.3%


In [ ]:
# Power curve visualization data
println("\nPower Curve:")
println("v [m/s]  P_avail [kW]  P_out [kW]  η [%]")
println("─"^45)
for v in [0, 3, 5, 8, 10, 12, 15, 20, 25, 30]
    r = analyze_wt(wt, Float64(v))
    println("$(lpad(v, 5))    $(lpad(round(r.E_wind_in/1000, digits=2), 8))     $(lpad(round(r.W_elec/1000, digits=2), 8))    $(lpad(round(r.η_en*100, digits=1), 5))")
end

## Export

Functions available for import:
- `WindTurbine` - Model struct (accepts A_rotor or D_rotor)
- `wind_power_available` - Available wind power (cubic law)
- `wt_power_output` - Electrical output with power curve
- `wt_efficiency` - Instantaneous efficiency
- `analyze_wt` - Complete analysis